# Water Quality Data Preparation (Target = Site_ID)


This data is Sourced by Department of the Interior, Water quality data for the Refuge collected by volunteers collected once every two weeks: Turbidity, pH, Dissolved oxygen (DO),Salinity & Temperature. Sampling will occur at designated locations in the following water bodies: the Bay, D-Pool (fishing pond), C-Pool, B-Pool and A-Pool.

Problem Statment: to categorize areas into Bay,A, B, C , D pool( which are differentiated by the water Quality). which will further detect changes in water quality early on, indicating potential contamination. This can help authorities take timely action to prevent the spread of contaminants and ensure the safety of the water supply. 

How will building this model will help: model can be trained to predict water quality based on various parameters such as pH, temperature, and turbidity.to understand the contamination of the water source.This can help in targeting resources such as testing, treatment, and monitoring efforts, to the areas that need them the most. This can save time, money, and effort in managing water quality.This can also help in reducing the incidence of water-borne diseases and ensure safe drinking water for all.

## 1.0 Setup

In [491]:
import pandas as pd
import numpy as np

In [492]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
# set random seed to ensure that results are repeatable
np.random.seed(1)

## 2.0 Load data 

In [493]:
df = pd.read_csv('BKB_WaterQualityData_2020084.csv')

In [494]:
df.head()

,Site_Id,Unit_Id,Read_Date,Salinity (ppt),Dissolved Oxygen (mg/L),pH (standard units),Secchi Depth (m),Water Depth (m),Water Temp (?C),Air Temp-Celsius,Air Temp (?F),Time (24:00),Field_Tech,DateVerified,WhoVerified,AirTemp (C),Year
0,Bay,NaN,1/3/1994,1.3,11.7,7.3,0.40,0.40,5.9,8.0,46.40,11:00,NaN,NaN,NaN,8.0,1994
1,Bay,NaN,1/31/1994,1.5,12.0,7.4,0.20,0.35,3.0,2.6,36.68,11:30,NaN,NaN,NaN,2.6,1994
2,Bay,NaN,2/7/1994,1.0,10.5,7.2,0.25,0.60,5.9,7.6,45.68,9:45,NaN,NaN,NaN,7.6,1994
3,Bay,NaN,2/23/1994,1.0,10.1,7.4,0.35,0.50,10.0,2.7,36.86,NaN,NaN,NaN,NaN,2.7,1994
4,Bay,NaN,2/28/1994,1.0,12.6,7.2,0.20,0.40,1.6,0.0,32.00,10:30,NaN,NaN,NaN,0.0,1994


## 3.0 Conduct initial exploration of the data

In [495]:
# How many rows and columns
df.shape

(2371, 17)

In [496]:
#How much of a problem do we have with na's?
df.isnull().sum()

Site_Id                       1
Unit_Id                    2339
Read_Date                     5
Salinity (ppt)              130
Dissolved Oxygen (mg/L)     851
pH (standard units)          95
Secchi Depth (m)             73
Water Depth (m)              71
Water Temp (?C)             121
Air Temp-Celsius           2286
Air Temp (?F)                71
Time (24:00)                 63
Field_Tech                   39
DateVerified               1918
WhoVerified                1918
AirTemp (C)                   0
Year                          0
dtype: int64

In [497]:
#What types of data are there?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2371 entries, 0 to 2370
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Site_Id                  2370 non-null   object 
 1   Unit_Id                  32 non-null     object 
 2   Read_Date                2366 non-null   object 
 3   Salinity (ppt)           2241 non-null   float64
 4   Dissolved Oxygen (mg/L)  1520 non-null   float64
 5   pH (standard units)      2276 non-null   float64
 6   Secchi Depth (m)         2298 non-null   float64
 7   Water Depth (m)          2300 non-null   float64
 8   Water Temp (?C)          2250 non-null   float64
 9   Air Temp-Celsius         85 non-null     float64
 10  Air Temp (?F)            2300 non-null   float64
 11  Time (24:00)             2308 non-null   object 
 12  Field_Tech               2332 non-null   object 
 13  DateVerified             453 non-null    object 
 14  WhoVerified             

In [498]:
# generate a statistical summary of the numeric value in the data
df.describe()

,Salinity (ppt),Dissolved Oxygen (mg/L),pH (standard units),Secchi Depth (m),Water Depth (m),Water Temp (?C),Air Temp-Celsius,Air Temp (?F),AirTemp (C),Year
count,2241.000000,1520.000000,2276.000000,2298.000000,2300.000000,2250.000000,85.000000,2300.000000,2371.000000,2371.000000
mean,0.717068,6.646263,7.168212,0.524898,0.762559,18.062138,16.437647,62.051637,15.663050,2006.049346
std,1.230819,2.506608,0.788485,0.473663,0.621140,8.298246,11.754138,15.492236,10.314720,9.002373
min,0.000000,0.000000,0.300000,0.000000,0.010000,0.000000,0.000000,10.500000,-17.777778,1899.000000
25%,0.000000,4.800000,6.500000,0.300000,0.400000,11.000000,9.000000,49.000000,8.888889,2001.000000
50%,0.000000,6.500000,7.000000,0.400000,0.650000,19.000000,15.000000,63.000000,16.666667,2007.000000
75%,1.000000,8.500000,7.500000,0.650000,0.950000,25.000000,21.700000,75.000000,23.888889,2012.000000
max,9.000000,15.100000,9.900000,9.000000,12.000000,74.000000,74.000000,92.300000,33.500000,2019.000000


In [499]:
## using summarytool
import summarytools
from summarytools import dfSummary
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Site_Id[object],1. Bay2. D3. B4. A5. C6. d7. nan,794 (33.5%)440 (18.6%)437 (18.4%)434 (18.3%)264 (11.1%)1 (0.0%)1 (0.0%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAChCAYAAADQgM1HAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACxElEQVR4nO3bzW0TUQBG0TeRxc/GVhQpJUAPKYJiKcJF0EMUZRYE2JgFS5AIirkv8pxTwbN0NaOx3recTqcBhavZB2A7xEZGbGTERkZsZJYxxvsxxpvZBzmjH6fT6Wn2Ifjd7ubm5tN+v7+efZBzWdf1YVmWz4J7fXb7/f767u7u6XA4fJt9mJd6fHx8dzwer+/v79+MMcT2yuzGGONwOHy7vb39OvswZ/J+9gH4Mx8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZHZj/LqaM/sg53Apv+NS7dZ1fTgej9fjQq7mrOv6MMb4Mfsc/M61cDKLRTyVWU82T58NmjJ4MUrZpnzwYpSyXbMGLxfx5cu/8acuGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRyddVFlDbNWVdZQG1TTYIZKyryHiykbGuImNdRca6iow/dcmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MgYvJAxeCHjWjgZgxcyPhDIzHiNeoVuVL6usqzarnRdZVm1bTPWVZZVG+UDgYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjUy6rrKs2rZ8XWVZtV2uhZOxriJzriebpxV/dZbBixELz/HiwYsRC891rsGLEQt/5U9dMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2Mi8evBix8FxnGbwYsfAcroWTMXgh4wOBjNjIXC3L8nFZFl+U/HdXY4wPY4y3sw/C5fMaJSM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjczXG+DLG+D77IFw+gxcyXqNkxEbG4IWMwQsZr1EyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiI2NdRca6iozXKBmxkREbGbGRERuZn5QvGPUqnIdlAAAAAElFTkSuQmCC"">",1(0.0%)
2,Unit_Id[object],1. nan2. 01csv3. 01CSV,"2,339 (98.7%)28 (1.2%)4 (0.2%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABoUlEQVR4nO3cvW3DMBRF4UtCcKKGgmDAIySdB/AQGTZDaACX2cEQzCJy3DBFHMMDRJcAc74JXnGgH0JPoZQiwCHWHgD/R5DUS9rUHgRNu5ZSlm673b6llMba06BdOec5hPDepZTGw+GwDMNwqT0U2nM+n5+naRpPp9Omk6RhGC673e6z9mBoVi/xggAjYoMNscGG2GBDbLAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22HTSzzdHtQdBmx7b6nLO8zRNo27fHAF/Lec8S7qygwCHayllCazyweXxynYtpSyV50HD7ttVvxswBIe1xJTSuN/vdVvn49kNq4mS1Pf9V+1B0D4OdWFDbLAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGmyhJy7I81R4E7Ys55/l4PN43YGoPhHaxgwAbtqtgwwsCbIgNNjGE8BpC4F8fWF2U9CKJow+sjtsobIgNNsQGG2KDDbHBhthgQ2ywITbYEBtsiA02xAYbYoMNscGG2GBDbLAhNthESR+S+DU9Vsd2FWy4jcLmG8RFXzQh/eqoAAAAAElFTkSuQmCC"">","2,339(98.7%)"
3,Read_Date[object],1. 4/4/20172. 11/20/20013. 9/7/20064. 3/9/20065. 3/15/20066. 4/6/20067. 5/5/20068. 5/17/20069. 6/1/200610. 7/6/200611. other,"6 (0.3%)5 (0.2%)5 (0.2%)5 (0.2%)5 (0.2%)5 (0.2%)5 (0.2%)5 (0.2%)5 (0.2%)5 (0.2%)2,320 (97.8%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADaElEQVR4nO3cQWoUYRRG0VehiWbSTQhkCbqHLMLFuohehHsIITUwMZNyIIo41lvQnjPK8B9ceDTUl2XbtoHC1d4P4P8hNjJiIyM2MmIjIzYyYiMjNjJiIyM2MlfLsnxcluX93g/h8l3NzIeZebf3Q7h8zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkbmamS8z823vh3D5Fv/FiIozSkZsZKyryFhXkXFGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIGLyQMXgh44ySERsZgxcyBi9knFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERkZsZKyryFhXkXFGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i44ySERsZgxcyBi9knFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i44ySERsZ6yoy1lVknFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjMELGYMXMs4oGbGRERsZ6yoy1lVknFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjLWVWSsq8g4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjMELGYMXMs4omWVmbmbmeu+HcLHetm17mZk53N3dfToej7d7v4jLtK7r07Isn7dtezkcj8fbh4eHl9Pp9Lr3w7gsz8/P78/n8+3j4+P1zLwcZmZOp9Pr/f39153fxmW6+fmHHwhkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkDjM/PgXZ+yFcnj+7Oqzr+nQ+n2/nt09B4G9Z1/VpZt5mfBbOv/frs3DrKjJ+IJARGxmxkREbGbGRERsZsZERGxmxkfkOCvhuJDOJMoEAAAAASUVORK5CYII="">",5(0.2%)
4,Salinity (ppt)[float64],Mean (sd) : 0.7 (1.2)min < med < max:0.0 < 0.0 < 9.0IQR (CV) : 1.

## Finding and procedure

## 4.0 Process the data

### 4.1  Conduct any data prepartion that should be done *BEFORE* the data split

#### Droping columns/variables we will not be using

In [500]:
df.drop('Unit_Id',axis=1,inplace=True)## droping Since 98% of the values are missing 

In [501]:
df.drop('DateVerified',axis=1,inplace=True) ## droping 91% of the values are missing 

In [502]:
df.drop('WhoVerified',axis=1,inplace=True) ## droping 91% of the values are missing 

In [503]:
df.drop('Field_Tech',axis=1,inplace=True) ## dropping since the names of people are not directly impact our predictor variable

In [504]:
df.drop('Air Temp-Celsius',axis=1,inplace=True) ## dropping since duplicate

In [505]:
df.drop('AirTemp (C)',axis=1,inplace=True) ## dropping since duplicate

In [506]:
df.drop('Time (24:00)',axis=1,inplace=True)

In [507]:
df.head()

,Site_Id,Read_Date,Salinity (ppt),Dissolved Oxygen (mg/L),pH (standard units),Secchi Depth (m),Water Depth (m),Water Temp (?C),Air Temp (?F),Year
0,Bay,1/3/1994,1.3,11.7,7.3,0.40,0.40,5.9,46.40,1994
1,Bay,1/31/1994,1.5,12.0,7.4,0.20,0.35,3.0,36.68,1994
2,Bay,2/7/1994,1.0,10.5,7.2,0.25,0.60,5.9,45.68,1994
3,Bay,2/23/1994,1.0,10.1,7.4,0.35,0.50,10.0,36.86,1994
4,Bay,2/28/1994,1.0,12.6,7.2,0.20,0.40,1.6,32.00,1994


### converting the read_date column into the data-time formate

In [508]:
df['Read_Date'] = pd.to_datetime(df['Read_Date'])

In [509]:
df['Month'] = df['Read_Date'].dt.month

In [510]:
df.drop('Read_Date',axis=1,inplace=True) ## dropping the Read_Date column as we have exacted the month and Year is already present as a column

In [511]:
df.head()

,Site_Id,Salinity (ppt),Dissolved Oxygen (mg/L),pH (standard units),Secchi Depth (m),Water Depth (m),Water Temp (?C),Air Temp (?F),Year,Month
0,Bay,1.3,11.7,7.3,0.40,0.40,5.9,46.40,1994,1.0
1,Bay,1.5,12.0,7.4,0.20,0.35,3.0,36.68,1994,1.0
2,Bay,1.0,10.5,7.2,0.25,0.60,5.9,45.68,1994,2.0
3,Bay,1.0,10.1,7.4,0.35,0.50,10.0,36.86,1994,2.0
4,Bay,1.0,12.6,7.2,0.20,0.40,1.6,32.00,1994,2.0


### Dropping all the nan values in the target variable

In [512]:
df = df[df['Site_Id'].notna()]

In [513]:
len(df) ## dropped the single null value 

2370

### we can see that one of the value D is marked as small d . we will convert this label

In [514]:
df['Site_Id'].value_counts() 

Bay    794
D      440
B      437
A      434
C      264
d        1
Name: Site_Id, dtype: int64

In [515]:
df['Site_Id'] = df['Site_Id'].replace(['d'], 'D')

In [516]:
df['Site_Id'].value_counts() 

Bay    794
D      441
B      437
A      434
C      264
Name: Site_Id, dtype: int64

### Converting the label 

In [517]:
labelencoder = LabelEncoder()

In [518]:
df['Site_Id'] = labelencoder.fit_transform(df['Site_Id'])

In [519]:
df['Site_Id'].value_counts()

2    794
4    441
1    437
0    434
3    264
Name: Site_Id, dtype: int64

### Safe the pre-split processed data to a csv file

In [520]:
df.to_csv('Water_Quality_presplit_processed.csv', index=False)

### 4.2 Split data (train/test)

In [521]:
# split the data into validation and training set
train_df, test_df = train_test_split(df, test_size=0.3)
target = 'Site_Id'
predictors = list(df.columns)
predictors.remove(target)

### 4.3  Conduct any data prepartion that should be done *AFTER* the data split

#### Impute missing values

In [522]:
df.isnull().sum()

Site_Id                      0
Salinity (ppt)             129
Dissolved Oxygen (mg/L)    850
pH (standard units)         94
Secchi Depth (m)            72
Water Depth (m)             70
Water Temp (?C)            120
Air Temp (?F)               70
Year                         0
Month                        4
dtype: int64

In [523]:
df['Site_Id'].value_counts()

2    794
4    441
1    437
0    434
3    264
Name: Site_Id, dtype: int64

In [524]:
train_df['Salinity (ppt)'].isnull().sum()

88

In [525]:
len(train_df)

1659

### Filling the null values for both the train and test data

In [526]:
### Salinity
## this will act as the fit_transform
train_df['Salinity (ppt)'] = train_df['Salinity (ppt)'].fillna(df.groupby('Site_Id')['Salinity (ppt)'].transform('mean'))
## this will act as the tranform
test_df['Salinity (ppt)'] = test_df['Salinity (ppt)'].fillna(df.groupby('Site_Id')['Salinity (ppt)'].transform('mean'))

In [527]:
# Dissolved Oxygen (mg/L)
train_df['Dissolved Oxygen (mg/L)'] = train_df['Dissolved Oxygen (mg/L)'].fillna(df.groupby('Site_Id')['Dissolved Oxygen (mg/L)'].transform('mean'))
test_df['Dissolved Oxygen (mg/L)'] = test_df['Dissolved Oxygen (mg/L)'].fillna(df.groupby('Site_Id')['Dissolved Oxygen (mg/L)'].transform('mean'))

In [528]:
# pH (standard units)
train_df['pH (standard units)'] = train_df['pH (standard units)'].fillna(df.groupby('Site_Id')['pH (standard units)'].transform('mean'))
test_df['pH (standard units)'] = test_df['pH (standard units)'].fillna(df.groupby('Site_Id')['pH (standard units)'].transform('mean'))

In [529]:
# Secchi Depth (m)
train_df['Secchi Depth (m)'] = train_df['Secchi Depth (m)'].fillna(df.groupby('Site_Id')['Secchi Depth (m)'].transform('mean'))
test_df['Secchi Depth (m)'] = test_df['Secchi Depth (m)'].fillna(df.groupby('Site_Id')['Secchi Depth (m)'].transform('mean'))

In [530]:
# Water Depth (m)
train_df['Water Depth (m)'] = train_df['Water Depth (m)'].fillna(df.groupby('Site_Id')['Water Depth (m)'].transform('mean'))
test_df['Water Depth (m)'] = test_df['Water Depth (m)'].fillna(df.groupby('Site_Id')['Water Depth (m)'].transform('mean'))

In [531]:
# Water Temp (?C)                        
train_df['Water Temp (?C)'] = train_df['Water Temp (?C)'].fillna(df.groupby('Site_Id')['Water Temp (?C)'].transform('mean'))
test_df['Water Temp (?C)'] = test_df['Water Temp (?C)'].fillna(df.groupby('Site_Id')['Water Temp (?C)'].transform('mean'))

In [532]:
# Air Temp (?F)
train_df['Air Temp (?F)'] = train_df['Air Temp (?F)'].fillna(df.groupby('Site_Id')['Air Temp (?F)'].transform('mean'))
test_df['Air Temp (?F)'] = test_df['Air Temp (?F)'].fillna(df.groupby('Site_Id')['Air Temp (?F)'].transform('mean'))

In [533]:
# Month                        
train_df['Month'] = train_df['Month'].fillna(df.groupby('Site_Id')['Month'].transform('mean'))
test_df['Month'] = test_df['Month'].fillna(df.groupby('Site_Id')['Month'].transform('mean'))

In [534]:
train_df.isnull().sum()

Site_Id                    0
Salinity (ppt)             0
Dissolved Oxygen (mg/L)    0
pH (standard units)        0
Secchi Depth (m)           0
Water Depth (m)            0
Water Temp (?C)            0
Air Temp (?F)              0
Year                       0
Month                      0
dtype: int64

In [535]:
test_df.isnull().sum()

Site_Id                    0
Salinity (ppt)             0
Dissolved Oxygen (mg/L)    0
pH (standard units)        0
Secchi Depth (m)           0
Water Depth (m)            0
Water Temp (?C)            0
Air Temp (?F)              0
Year                       0
Month                      0
dtype: int64

#### From the both the train and training data we have imputed the null values using the means of the respected class varaiables which will helps us to reduce the data leak.

## Standardize numeric values

In [536]:
df.columns

Index(['Site_Id', 'Salinity (ppt)', 'Dissolved Oxygen (mg/L)',
       'pH (standard units)', 'Secchi Depth (m)', 'Water Depth (m)',
       'Water Temp (?C)', 'Air Temp (?F)', 'Year', 'Month'],
      dtype='object')

In [537]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = ['Salinity (ppt)', 'Dissolved Oxygen (mg/L)',
       'pH (standard units)', 'Secchi Depth (m)', 'Water Depth (m)',
       'Water Temp (?C)', 'Air Temp (?F)','Year', 'Month']                
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array


test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object

## Addressing class imbalance 

In [539]:
train_X = train_df[predictors]
train_y = train_df[target] 
test_X = test_df[predictors]
test_yd = test_df[target]

In [544]:
import imblearn
from collections import Counter

In [547]:
train_y.value_counts()

2    563
4    312
0    310
1    300
3    174
Name: Site_Id, dtype: int64

In [545]:
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')

X_over, y_over = oversample.fit_resample(train_X, train_y)

print(Counter(y_over))

Counter({2: 563, 3: 563, 4: 312, 0: 310, 1: 300})


In [548]:
X_over.to_csv('Water_Quality_train_x.csv', index=False)
y_over.to_csv('Water_Quality_train_y.csv', index=False)
test_X.to_csv('Water_Quality_test_x.csv', index=False)
test_y.to_csv('Water_Quality_test_y.csv', index=False)